In [ ]:
import time
from selenium import webdriver
import pandas as pd
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException


driver = webdriver.Chrome('/Users/LG/Desktop/chromedriver/chromedriver') #웹드라이버 객체 지정
driver.maximize_window() #전체 화면으로 브라우저 창 확장

driver.get('http://atemshop.com/board/review/4/?undefined-scroll_top=-185.7777862548828%40crema-products-1&undefined-page=5&undefined-sort=128&undefined-crema-products-1-page=6&undefined-crema-products-1-sort=1&crema-products-1-page=7&crema-products-1-sort=128')
driver.switch_to.frame(driver.find_element_by_css_selector('#crema-products-1'))

#리뷰란 웹엘리먼트 
review_links = driver.find_elements_by_css_selector('#content > div > div.products_index_gallery__body > div.products_index_gallery__content > div > div.products_index_gallery__products_column.products_index_gallery__products_column_0 > div > div > a')

#제품 페이지 url
urls = []

for i in range(len(review_links)):
    urls.append(review_links[i].get_attribute('href'))

urls = urls[:7] + urls[9:15]

df_data = pd.DataFrame()

for url in urls:
    try:
        driver.get(url+'#prdReview') #페이지의 리뷰 섹션으로 이동
        time.sleep(1)
        # driver.switch_to.frame(driver.find_element_by_css_selector('#crema-product-reviews-2'))
        productCode = 000
        productName = driver.find_element_by_css_selector('#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.infoArea > div.headingArea > h2').text.strip()
        print(productName) #제품이름
        for i in range(20):
            try:
                time.sleep(1)
                driver.switch_to.window(driver.window_handles[0])
                driver.switch_to.frame(driver.find_element_by_css_selector('#crema-product-reviews-2'))
                next_butt = driver.find_element_by_css_selector('#content > div > div.widget_reviews.js-pagination-list > div.products_reviews__footer > div > div > a.pagination__button.pagination__button--next')
                reviews = driver.find_elements_by_css_selector('#content > div > div.widget_reviews.js-pagination-list > div.widget_reviews__body.products_reviews_list__body > div > ul > li')
                i = 0
                print(i)
                
                #제품 리뷰들 크롤링
                for review in reviews:
                    if review == reviews[i]:
                        print('잘 돌고 있다.')
                    i +=1
                    print(i)
                    temp_row = {'mall': 'atem', 'keyword': 'labnosh', 'productname': productName, 'options': None, 'productCode': productCode, 'score': None,'date': None, 'user_id': None, 'age': None, 'gender': None, 'main_text': None}
                    while True:
                        try:
                            more = review.find_element_by_css_selector('div > div.products_reviews_list_review__lcontents > div.products_reviews_list_review__content.review_content > div.products_reviews_list_review__content_inner.review_content__collapsed > div.products_reviews_list_review__message > a')
                            more.click()
                            time.sleep(1)
                            break
                        except WebDriverException:
                            driver.switch_to.window(driver.window_handles[0])
                            driver.execute_script("window.scrollBy(0,30)")
                            driver.switch_to.frame(driver.find_element_by_css_selector('#crema-product-reviews-2'))
                            time.sleep(0.5)

                    print('여기까지 OK')

                    # 날짜
                    date = review.find_element_by_css_selector('div > div.products_reviews_list_review__info_container > ul > li:nth-child(2) > div.products_reviews_list_review__info_value').text.strip()
                    temp_row['date'] = date
                    print(date)

                    # 아이디
                    id = review.find_element_by_css_selector('div > div.products_reviews_list_review__info_container > ul > li:nth-child(1) > div.products_reviews_list_review__info_value').text.strip()
                    temp_row['user_id'] = id
                    print(id)
                    
                    #옵션의 경우 아예 존재하지 않거나 나이,성별,제품옵션 등이 각각의 조합으로 있었음.
                    #옵션의 길이를 구해 각 옵션 조합별 정보를 크롤링함.
                    option_num = review.find_elements_by_css_selector('div > div.products_reviews_list_review__lcontents > div.products_reviews_list_review__content.review_content.review_content--expanded > div.products_reviews_list_review__content_inner.review_content__expanded > div.products_reviews_list_review__message > div.review_options > div')

                    if len(option_num) == 0:
                        temp_row['age'] = ''
                        temp_row['gender']=''
                        temp_row['options']=''

                        main_text = review.find_element_by_css_selector('div > div.products_reviews_list_review__lcontents > div.products_reviews_list_review__content.review_content.review_content--expanded > div.products_reviews_list_review__content_inner.review_content__expanded > div.products_reviews_list_review__message > div').text.strip()
                        temp_row['main_text'] = main_text

                    elif len(option_num) == 1:
                        options = review.find_elements_by_css_selector('div > div.products_reviews_list_review__lcontents > div.products_reviews_list_review__content.review_content.review_content--expanded > div.products_reviews_list_review__content_inner.review_content__expanded > div.products_reviews_list_review__message > div.review_options > div > div:nth-child(2) > span')
                        temp_row['options']=[]
                        for k in options:
                            option = k.find_element_by_css_selector('span:nth-child(2)').text.strip()
                            temp_row['options'].append(option)
                        temp_row['age'] = ''
                        temp_row['gender'] = ''
                        main_text = review.find_element_by_css_selector('div > div.products_reviews_list_review__lcontents > div.products_reviews_list_review__content.review_content.review_content--expanded > div.products_reviews_list_review__content_inner.review_content__expanded > div.products_reviews_list_review__message > div.products_reviews_list_review__message_content').text.strip()
                        temp_row['main_text'] = main_text


                    elif len(option_num) == 2:
                        # 나이
                        age = option_num[0].find_element_by_css_selector('div:nth-child(2)').text.strip()
                        temp_row['age'] = age

                        # 성별
                        gender = option_num[1].find_element_by_css_selector('div:nth-child(2)').text.strip()
                        temp_row['gender'] = gender

                        temp_row['options'] = ''
                        main_text = review.find_element_by_css_selector('div > div.products_reviews_list_review__lcontents > div.products_reviews_list_review__content.review_content.review_content--expanded > div.products_reviews_list_review__content_inner.review_content__expanded > div.products_reviews_list_review__message > div.products_reviews_list_review__message_content').text.strip()
                        temp_row['main_text'] = main_text

                    elif len(option_num) == 3:
                        # 나이
                        age = option_num[0].find_element_by_css_selector('div:nth-child(2)').text.strip()
                        temp_row['age'] = age

                        # 성별
                        gender = option_num[1].find_element_by_css_selector('div:nth-child(2)').text.strip()
                        temp_row['gender'] = gender

                        # 옵션
                        options = option_num[2].find_elements_by_css_selector('div:nth-child(2) > span')
                        temp_row['options']=[]
                        for k in options:
                            option = k.find_element_by_css_selector('span:nth-child(2)').text.strip()
                            temp_row['options'].append(option)

                        main_text = review.find_element_by_css_selector('div > div.products_reviews_list_review__lcontents > div.products_reviews_list_review__content.review_content.review_content--expanded > div.products_reviews_list_review__content_inner.review_content__expanded > div.products_reviews_list_review__message > div.products_reviews_list_review__message_content').text.strip()
                        temp_row['main_text'] = main_text


                    # 별점
                    y = review.find_elements_by_css_selector('div > div.products_reviews_list_review__lcontents > div.score.products_reviews_list_review__head > div > div.products_reviews_list_review__score_star_rating > span.star--empty')
                    temp_row['score'] = 5-len(y) #별점에서 별의 길이를 숫자로 가져옴
                    print(5-len(y))

                    print(temp_row)

                    df_data = df_data.append(temp_row, ignore_index=True)

                while True:
                    try: #다음 리뷰 페이지로 이동
                        driver.switch_to.window(driver.window_handles[0])
                        driver.switch_to.frame(driver.find_element_by_css_selector('#crema-product-reviews-2'))
                        time.sleep(1)
                        next_butt.click()
                        break
                    except WebDriverException: #리뷰란 페이지 스크롤 이동
                        driver.switch_to.window(driver.window_handles[0])
                        driver.execute_script("window.scrollBy(0,600)")

            except StaleElementReferenceException: #엘리먼트가 더 이상 없을때 break
                break

    except NoSuchElementException: #엘리먼트가 더 이상 없을때 break
        break

#csv로 결과 리턴        
df_data.to_csv('labnosh1.csv',encoding='utf-8',index=False)